In [ ]:
# Perform git clone as follows

!git clone https://github.com/WEIRDLabUW/cse542_sp24_hw2.git
!cp -r cse542_sp24_hw2/* .

# !NOTE!: Once you are done, copy your implementation of policy gradient, actor critic and
# in the notebook here back to the python script
# when submiting your code

!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf
!pip install setuptools==65.5.0 "wheel<0.40.0"
!pip install gym==0.19.0
!pip install gymnasium==0.29.1
!pip install gymnasium-robotics[mujoco-py]
!pip install gym-notices==0.0.8
!pip install matplotlib
!pip install mujoco
!pip install free-mujoco-py
!pip install pybullet
import os
os.environ['LD_PRELOAD']=':/usr/lib/x86_64-linux-gnu/libGLEW.so'
import os
import torch
import numpy as np
from torch import nn
from torch import optim
from IPython.display import clear_output
import argparse
import collections
import functools
import math
import time
import random
from typing import Any, Callable, Dict, Optional, Sequence, List
import gym
import mujoco_py
from gym import utils
from gym.envs.mujoco import mujoco_env
import torch.nn.functional as F
import copy
from typing import Tuple, Optional, Union
import matplotlib.pyplot as plt


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


Cloning into 'cse542_sp24_hw2'...
fatal: could not read Username for 'https://github.com': No such device or address
cp: cannot stat 'cse542_sp24_hw2/*': No such file or directory
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libglew-dev is already the newest version (2.2.0-4).
libgl1-mesa-dev is already the newest version (23.2.1-1ubuntu3.1~22.04.2).
libosmesa6-dev is already the newest version (23.2.1-1ubuntu3.1~22.04.2).
software-properties-common is already the newest version (0.99.22.9).
libgl1-mesa-glx is already the newest version (23.0.4-0ubuntu1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
patchelf is already the newest version (0.14.3-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
  Using cached gym-0.19.0.tar.gz (1.6 MB)
  Preparing metadata (setup.py) ... done
  Using cached clou

KeyboardInterrupt: 

In [ ]:

import torch
import torch.nn as nn

import math
import copy
import numpy as np
import pickle
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch import distributions as pyd
import torch.optim as optim
from torch.distributions import Categorical



def log_density(x, mu, std, logstd):
    var = std.pow(2)
    log_density = -(x - mu).pow(2) / (2 * var) \
                  - 0.5 * math.log(2 * math.pi) - logstd
    return log_density.sum(1, keepdim=True)


class PGPolicy(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_dim=64, hidden_depth=2):
        super(PGPolicy, self).__init__()
        self.trunk = mlp(num_inputs, hidden_dim, num_outputs*2, hidden_depth)

    def forward(self, x):
        outs = self.trunk(x)
        mu, std, log_std = self.dist_create(outs)
        std = torch.exp(log_std)
        action = torch.normal(mu, std)
        self.mu = mu

        return action, std, log_std

    def dist_create(self, logits):
        min_log_std = -5
        max_log_std = 5
        loc, scale = torch.split(logits, logits.shape[-1] // 2, dim=-1)
        loc = torch.tanh(loc)

        log_std = torch.sigmoid(scale)
        log_std = min_log_std + log_std * (max_log_std - min_log_std)
        std = torch.exp(log_std)
        return loc, std, log_std


class PGBaseline(nn.Module):
    def __init__(self, num_inputs, hidden_dim=64, hidden_depth=2):
        super(PGBaseline, self).__init__()
        self.trunk = mlp(num_inputs, hidden_dim, 1, hidden_depth)

    def forward(self, x):
        v = self.trunk(x)
        return v

class ACPolicy(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_dim=64, hidden_depth=2):
        super(ACPolicy, self).__init__()
        self.trunk = mlp(num_inputs, hidden_dim, num_outputs*2, hidden_depth)
        self.min_log_std = -5
        self.max_log_std  = 5

    def forward(self, x):
        outs = self.trunk(x)
        mu, std, log_std = self.dist_create(outs)
        action = self.dist_sample_no_postprocess(mu, std)
        self.std = std
        self.mu = mu
        return action, std, log_std

    def dist_create(self, logits):
        loc, scale = torch.split(logits, logits.shape[-1] // 2, dim=-1)
        loc = torch.tanh(loc)

        log_std = torch.sigmoid(scale)
        log_std = self.min_log_std + log_std * (self.max_log_std - self.min_log_std)
        std = torch.exp(log_std)
        return loc, std, log_std

    def dist_entropy(self):
        log_normalized = 0.5 * math.log(2 * math.pi) + torch.log(self.std)
        entropy = 0.5 + log_normalized
        entropy = entropy * torch.ones_like(self.mu)
        return entropy.sum(dim=-1)

    def dist_sample_no_postprocess(self, mu, std):
        action = torch.zeros((mu.shape[0], 1)).to(device)
        # TODO START
        # Hint: reparameterization trick
        # TODO END
        return action



class QF(nn.Module):
    def __init__(self, num_inputs, hidden_dim=64, hidden_depth=2):
        super(QF, self).__init__()
        self.trunk = mlp(num_inputs, hidden_dim, 1, hidden_depth)

    def forward(self, x):
        v = self.trunk(x)
        return v



class TargetQF(nn.Module):
    def __init__(self, num_inputs, hidden_dim=64, hidden_depth=2):
        super(TargetQF, self).__init__()
        self.trunk = mlp(num_inputs, hidden_dim, 1, hidden_depth)

    def forward(self, x):
        v = self.trunk(x)
        return v


def collect_trajs(
        env,
        agent,
        replay_buffer,
        device,
        episode_length=math.inf,
        render=False,
):
    # Collect the following data
    raw_obs = []
    raw_next_obs = []
    actions = []
    rewards = []
    dones = []
    images = []

    path_length = 0

    o = env.reset()
    if render:
        env.render()

    while path_length < episode_length:
        o_for_agent = o

        action, _, _ = agent(torch.Tensor(o_for_agent).unsqueeze(0).to(device))
        action= action.cpu().detach().numpy()[0]

        # Step the simulation forward
        next_o, r, done, env_info = env.step(copy.deepcopy(action))

        replay_buffer.add(o,
                          action,
                          r,
                          next_o,
                          done)

        # Render the environment
        if render:
            env.render()

        raw_obs.append(o)
        raw_next_obs.append(next_o)
        actions.append(action)
        rewards.append(r)
        dones.append(done)
        path_length += 1
        if done:
            break
        o = next_o

    # Prepare the items to be returned
    observations = np.array(raw_obs)
    next_observations = np.array(raw_next_obs)
    actions = np.array(actions)
    if len(actions.shape) == 1:
        actions = np.expand_dims(actions, 1)
    rewards = np.array(rewards)
    if len(rewards.shape) == 1:
        rewards = rewards.reshape(-1, 1)
    dones = np.array(dones).reshape(-1, 1)

    # Return in the following format
    return dict(
        observations=observations,
        next_observations=next_observations,
        actions=actions,
        rewards=rewards,
        dones=np.array(dones).reshape(-1, 1),
        images=np.array(images)
    )



def mlp(input_dim, hidden_dim, output_dim, hidden_depth, output_mod=None):
    if hidden_depth == 0:
        mods = [nn.Linear(input_dim, output_dim)]
    else:
        mods = [nn.Linear(input_dim, hidden_dim), nn.ReLU(inplace=True)]
        for i in range(hidden_depth - 1):
            mods += [nn.Linear(hidden_dim, hidden_dim), nn.ReLU(inplace=True)]
        mods.append(nn.Linear(hidden_dim, output_dim))
    if output_mod is not None:
        mods.append(output_mod)
    trunk = nn.Sequential(*mods)
    return trunk




def rollout(
        env,
        agent,
        episode_length=math.inf,
        render=False,
):
    # Collect the following data
    raw_obs = []
    raw_next_obs = []
    actions = []
    rewards = []
    dones = []
    images = []

    entropy = None
    log_prob = None
    agent_info = None
    path_length = 0

    o = env.reset()
    if render:
        env.render()

    while path_length < episode_length:
        o_for_agent = o

        action, _, _ = agent(torch.Tensor(o_for_agent).unsqueeze(0).to(device))
        action = action.cpu().detach().numpy()[0]

        # Step the simulation forward
        next_o, r, done, env_info = env.step(copy.deepcopy(action))

        # Render the environment
        if render:
            env.render()

        raw_obs.append(o)
        raw_next_obs.append(next_o)
        actions.append(action)
        rewards.append(r)
        dones.append(done)
        path_length += 1
        if done:
            break
        o = next_o

    # Prepare the items to be returned
    observations = np.array(raw_obs)
    next_observations = np.array(raw_next_obs)
    actions = np.array(actions)
    if len(actions.shape) == 1:
        actions = np.expand_dims(actions, 1)
    rewards = np.array(rewards)
    if len(rewards.shape) == 1:
        rewards = rewards.reshape(-1, 1)
    dones = np.array(dones).reshape(-1, 1)

    # Return in the following format
    return dict(
        observations=observations,
        next_observations=next_observations,
        actions=actions,
        rewards=rewards,
        dones=np.array(dones).reshape(-1, 1),
        images=np.array(images)
    )


In [ ]:
class ReplayBuffer(object):
    """Buffer to store environment transitions."""

    def __init__(self, obs_size, action_size, capacity, device):
        self.capacity = capacity
        self.device = device

        self.obses = np.empty((capacity, obs_size), dtype=np.float32)
        self.next_obses = np.empty((capacity, obs_size), dtype=np.float32)
        self.actions = np.empty((capacity, action_size), dtype=np.float32)
        self.rewards = np.empty((capacity, 1), dtype=np.float32)
        self.not_dones = np.empty((capacity, 1), dtype=np.float32)

        self.idx = 0
        self.last_save = 0
        self.full = False

    def __len__(self):
        return self.capacity if self.full else self.idx

    def add(self, obs, action, reward, next_obs, done):
        idxs = np.arange(self.idx, self.idx + obs.shape[0]) % self.capacity
        self.obses[idxs] = copy.deepcopy(obs)
        self.actions[idxs] = copy.deepcopy(action)
        self.rewards[idxs] = copy.deepcopy(reward)
        self.next_obses[idxs] = copy.deepcopy(next_obs)
        self.not_dones[idxs] = 1.0 - copy.deepcopy(done)

        self.full = self.full or (self.idx + obs.shape[0] >= self.capacity)
        self.idx = (self.idx + obs.shape[0]) % self.capacity

    def sample(self, batch_size):
        idxs = np.random.randint(0,
                                 self.capacity if self.full else self.idx,
                                 size=batch_size)
        obses = torch.as_tensor(self.obses[idxs], device=self.device).float()
        actions = torch.as_tensor(self.actions[idxs], device=self.device)
        rewards = torch.as_tensor(self.rewards[idxs], device=self.device)
        next_obses = torch.as_tensor(self.next_obses[idxs],
                                     device=self.device).float()
        not_dones = torch.as_tensor(self.not_dones[idxs], device=self.device)

        return obses, actions, rewards, next_obses, not_dones



def compute_losses(policy, qf, target_qf, obs_t, actions_t, rewards_t, next_obs_t, not_dones_t, device, discount=0.99):
    policy_loss = torch.Tensor(np.array([0])).to(device)
    qf_loss = torch.Tensor(np.array([0])).to(device)


    obs_t = torch.Tensor(obs_t).to(device)
    actions_t = torch.Tensor(actions_t).to(device)
    rewards_t = torch.Tensor(rewards_t).to(device)
    next_obs_t = torch.Tensor(next_obs_t).to(device)
    not_dones_t = torch.Tensor(not_dones_t).to(device)
    # TODO START
    # Hint: compute policy_loss and qf_loss.
    # TODO END

    return policy_loss, qf_loss


def soft_update_target(net, target_net, tau):
    for param, target_param in zip(net.parameters(), target_net.parameters()):
        target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)


def simulate_policy_ac(
        env,
        policy,
        qf,
        target_qf,
        replay_buffer,
        device,
        episode_length: int = 100,
        num_epochs: int = 200,
        batch_size=32,
        target_weight= 5e-3,
        num_update_steps=100,
        render = False,
        print_freq=10,
        learning_rate = 3e-4,
):
    env.reset()

    policy_optimizer = optim.Adam(policy.parameters(), lr=learning_rate)
    qf_optimizer = optim.Adam(qf.parameters(), lr=learning_rate)

    # Copy parameters initially
    soft_update_target(qf, target_qf, 1.0)

    for iter_num in range(num_epochs):
        sample_trajs = []

        # Sampling trajectories
        for it in range(batch_size):
            sample_traj = collect_trajs(env, policy, replay_buffer, device, episode_length=episode_length, render=render)
            sample_trajs.append(sample_traj)

        if iter_num % print_freq == 0:
            rewards_np = np.mean(np.asarray([traj['rewards'].sum() for traj in sample_trajs]))
            path_length = np.max(np.asarray([traj['rewards'].shape[0] for traj in sample_trajs]))
            print("Episode: {}, reward: {}, max path length: {}".format(iter_num, rewards_np, path_length))

        for update_num in range(num_update_steps):
            obs_t, actions_t, rewards_t, next_obs_t, not_dones_t = replay_buffer.sample(batch_size)

            policy_loss, qf_loss = compute_losses(policy, qf, target_qf, obs_t, actions_t, rewards_t, next_obs_t,
                                                  not_dones_t, device)

            policy_optimizer.zero_grad()
            policy_loss.backward()
            policy_optimizer.step()

            qf_optimizer.zero_grad()
            qf_loss.backward()
            qf_optimizer.step()


            soft_update_target(qf, target_qf, target_weight)


In [ ]:
def train_model(policy, baseline, trajs, policy_optim, baseline_optim, device, gamma=0.99, baseline_train_batch_size=64,
                baseline_num_epochs=5):
    # Fill in your policy gradient implementation here
    states_all = []
    actions_all = []
    returns_all = []
    for traj in trajs:
        states_singletraj = traj['observations']
        actions_singletraj = traj['actions']
        rewards_singletraj = traj['rewards']
        returns_singletraj = np.zeros_like(rewards_singletraj)
        # TODO START

        # TODO: Compute the returns on the current batch of trajectories
        # Hint: Go through all the trajectories in trajs and compute their return to go: discounted sum of rewards from that timestep to the end.
        # Hint: This is easy to do if you go backwards in time and sum up the reward as a running sum.
        # Hint: Remember that return to go is return = r[t] + gamma*r[t+1] + gamma^2*r[t+2] + ...

        # TODO END
        states_all.append(states_singletraj)
        actions_all.append(actions_singletraj)
        returns_all.append(returns_singletraj)
    states = np.concatenate(states_all)
    actions = np.concatenate(actions_all)
    returns = np.concatenate(returns_all)

    # TODO: Normalize the returns by subtracting mean and dividing by std
    # TODO START
    # Hint: Just do return - return.mean()/ (return.std() + EPS), where EPS is a small constant for numerics
    # TODO END


    criterion = torch.nn.MSELoss()
    n = len(states)
    arr = np.arange(n)
    for epoch in range(baseline_num_epochs):
        np.random.shuffle(arr)
        for i in range(n // baseline_train_batch_size):
            batch_index = arr[baseline_train_batch_size * i: baseline_train_batch_size * (i + 1)]
            batch_index = torch.LongTensor(batch_index).to(device)
            # TODO START
            # TODO: Train baseline by regressing onto returns
            # Hint: Regress the baseline from each state onto the above computed return to go. You can use similar code to behavior cloning to do so.
            # Hint: Iterate for baseline_num_epochs with batch size = baseline_train_batch_size
            # TODO END
            baseline_optim.zero_grad()
            loss.backward()
            baseline_optim.step()


    action, std, logstd = policy(torch.Tensor(states).to(device))
    log_policy = log_density(torch.Tensor(actions).to(device), policy.mu, std, logstd)
    baseline_pred = baseline(torch.from_numpy(states).float().to(device))
    # TODO START

    # TODO: Train policy by optimizing surrogate objective: -log prob * (return - baseline)
    # Hint: Policy gradient is given by: \grad log prob(a|s)* (return - baseline)
    # Hint: Return is computed above, you can computer log_probs using the log_density function imported.
    # Hint: You can predict what the baseline outputs for every state.
    # Hint: Then simply compute the surrogate objective by taking the objective as -log prob * (return - baseline)
    # Hint: You can then use standard pytorch machinery to take *one* gradient step on the policy

    # TODO END

    policy_optim.zero_grad()
    loss.backward()
    policy_optim.step()

    del states, actions, returns, states_all, actions_all, returns_all


# Training loop for policy gradient
def simulate_policy_pg(env, policy, baseline, num_epochs=200, max_path_length=200, batch_size=100,
                       gamma=0.99, baseline_train_batch_size=64, baseline_num_epochs=5, print_freq=10, device = "cuda", render=False):
    policy_optim = optim.Adam(policy.parameters())
    baseline_optim = optim.Adam(baseline.parameters())

    for iter_num in range(num_epochs):
        sample_trajs = []

        # Sampling trajectories
        for it in range(batch_size):
            sample_traj = rollout(
                env,
                policy,
                episode_length=max_path_length,
                render=False)
            sample_trajs.append(sample_traj)

        # Logging returns occasionally
        if iter_num % print_freq == 0:
            rewards_np = np.mean(np.asarray([traj['rewards'].sum() for traj in sample_trajs]))
            path_length = np.max(np.asarray([traj['rewards'].shape[0] for traj in sample_trajs]))
            print("Episode: {}, reward: {}, max path length: {}".format(iter_num, rewards_np, path_length))

        # Training model
        train_model(policy, baseline, sample_trajs, policy_optim, baseline_optim, device, gamma=gamma,
                    baseline_train_batch_size=baseline_train_batch_size, baseline_num_epochs=baseline_num_epochs)



In [ ]:
class Args:
    def __init__(self, task, test=False, render=False):
        self.task = task # 'behavior_cloning' or 'dagger'
        self.test = test # whether test only
        self.render = render # whether to render during test
args = Args('policy_gradient', False,  False)

In [ ]:
def evaluate(env, policy, num_validation_runs=10, episode_length=200, render=False):
    success_count = 0
    rewards_suc = 0
    rewards_all = 0
    for k in range(num_validation_runs):
        o = env.reset()
        path = rollout(
                env,
                policy,
                episode_length=episode_length,
                render=render)
        success = len(path['dones']) == episode_length

        if success:
            success_count += 1
            rewards_suc += np.sum(path['rewards'])
        rewards_all += np.sum(path['rewards'])
        print(f"test {k}, success {success}, reward {np.sum(path['rewards'])}")
    print("Success rate: ", success_count/num_validation_runs)
    print("Average reward (success only): ", rewards_suc/max(success_count, 1))
    print("Average reward (all): ", rewards_all/num_validation_runs)

In [ ]:
import random

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

if __name__ == '__main__':
    env = gym.make("InvertedPendulum-v2")
    episode_length = 200
    if args.task == 'policy_gradient':
        # Define policy and value function
        hidden_dim_pol = 64
        hidden_depth_pol = 2
        hidden_dim_baseline = 64
        hidden_depth_baseline = 2
        policy = PGPolicy(env.observation_space.shape[0], env.action_space.shape[0], hidden_dim=hidden_dim_pol, hidden_depth=hidden_depth_pol)
        baseline = PGBaseline(env.observation_space.shape[0], hidden_dim=hidden_dim_baseline, hidden_depth=hidden_depth_baseline)
        policy.to(device)
        baseline.to(device)

        # Training hyperparameters
        num_epochs=200
        max_path_length=200
        batch_size=64
        gamma=0.99
        baseline_train_batch_size=64
        baseline_num_epochs=5
        print_freq=10

        if not args.test:
            # Train policy gradient
            simulate_policy_pg(env, policy, baseline, num_epochs=num_epochs, max_path_length=episode_length, batch_size=batch_size,
                            gamma=gamma, baseline_train_batch_size=baseline_train_batch_size, device = device, baseline_num_epochs=baseline_num_epochs, print_freq=print_freq, render=args.render)
            torch.save(policy.state_dict(), 'pg_final.pth')
        else:
            print('loading pretrained pg')
            policy.load_state_dict(torch.load(f'pg_final.pth'))


    if args.task == 'actor_critic':
        # Define replay buffer
        hidden_dim = 64
        hidden_depth = 2

        obs_size = env.observation_space.shape[0]
        ac_size = env.action_space.shape[0]

        capacity=10000
        replay_buffer = ReplayBuffer(obs_size,
                                     ac_size,
                                     capacity,
                                     device)


        num_epochs = 200
        discount = 0.99
        print_freq = 10
        num_update_steps = 100
        batch_size = 64

        policy = ACPolicy(env.observation_space.shape[0], env.action_space.shape[0], hidden_dim=hidden_dim, hidden_depth=hidden_depth).to(device)

        if not args.test:
            # Train actor critic
            qf = QF(env.observation_space.shape[-1] + env.action_space.shape[-1], hidden_dim=hidden_dim, hidden_depth=hidden_depth).to(device)
            target_qf = TargetQF(env.observation_space.shape[-1] + env.action_space.shape[-1], hidden_dim=hidden_dim,
                                   hidden_depth=hidden_depth).to(device)
            simulate_policy_ac(env, policy, qf, target_qf, replay_buffer, device,
                               episode_length=episode_length,
                               num_epochs=num_epochs, batch_size=batch_size, num_update_steps=num_update_steps,
                               print_freq=print_freq,
                               render=args.render)
            torch.save(policy.state_dict(), 'ac_final.pth')
        else:
            print('loading pretrained ac')
            policy.load_state_dict(torch.load(f'ac_final.pth'))

NameError: name 'torch' is not defined

In [ ]:
evaluate(env, policy,  num_validation_runs=100, episode_length=episode_length, render=args.render)

test 0, success True, reward 200.0
test 1, success False, reward 165.0
test 2, success True, reward 200.0
test 3, success True, reward 200.0
test 4, success False, reward 129.0
test 5, success False, reward 161.0
test 6, success True, reward 200.0
test 7, success True, reward 200.0
test 8, success False, reward 144.0
test 9, success False, reward 173.0
test 10, success True, reward 200.0
test 11, success True, reward 200.0
test 12, success True, reward 200.0
test 13, success False, reward 117.0
test 14, success False, reward 143.0
test 15, success False, reward 158.0
test 16, success True, reward 200.0
test 17, success False, reward 189.0
test 18, success False, reward 143.0
test 19, success True, reward 200.0
test 20, success True, reward 200.0
test 21, success True, reward 200.0
test 22, success True, reward 200.0
test 23, success True, reward 200.0
test 24, success False, reward 140.0
test 25, success False, reward 182.0
test 26, success True, reward 200.0
test 27, success True, rew